In [ ]:
!pip install kaggle

**Importing the Dependencies**

In [ ]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection- Kaggle API**

In [ ]:
kaggle_dictionary = json.load(open("/content/kaggle (1).json"))

In [ ]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [ ]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [ ]:
!ls

 imdb-dataset-of-50k-movie-reviews.zip	'kaggle (1).json'   sample_data


In [ ]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls

'IMDB Dataset.csv'			'kaggle (1).json'
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


**Loading teh Dataset**

In [ ]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()
data.tail(10)

,review,sentiment
49990,"Lame, lame, lame!!! A 90-minute cringe-fest th...",negative
49991,"Les Visiteurs, the first movie about the medie...",negative
49992,John Garfield plays a Marine who is blinded by...,positive
49993,Robert Colomb has two full-time jobs. He's kno...,negative
49994,This is your typical junk comedy.<br /><br />T...,negative
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
#data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [ ]:
data.replace({"sentiment":  {"positive" :2, "negative":5}},inplace=True)

NameError: name 'plt' is not defined

In [ ]:
data.isnull().sum()

,review,sentiment
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
49995,False,False
49996,False,False
49997,False,False
49998,False,False


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
print(data['sentiment'][15200])

0


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(test_data['review'][15200])

As far as films go, this is likable enough. Entertaining characters, good dialogue, interesting enough story. I would have really quite liked it had I not been irritated immensely whilst watching at the utter disrespect it shows the city it is set in.<br /><br />Glasgow. In Scotland. Yet every character is English (save for Sean's girlfriend, who is Dutch). Scottish accents are heard only fleetingly in menial jobs & roles. As a Scottish woman (& as a viewer who likes her "real life" films to be a bit more like real life) I really don't think it would have hurt to use any one of the countless talented Scottish actors...or at least got English ones who could toss together a decent accent! The futile attempt at using the word "wee" a few times did nothing but to further the insult.


In [ ]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [ ]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [ ]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [ ]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short-Term Memory**

In [ ]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Training the Model**

In [ ]:
model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 395ms/step - accuracy: 0.7248 - loss: 0.5270 - val_accuracy: 0.8461 - val_loss: 0.3605
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 188s 376ms/step - accuracy: 0.8605 - loss: 0.3389 - val_accuracy: 0.8012 - val_loss: 0.4287
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 211s 394ms/step - accuracy: 0.8772 - loss: 0.3060 - val_accuracy: 0.8691 - val_loss: 0.3208
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 394ms/step - accuracy: 0.8948 - loss: 0.2640 - val_accuracy: 0.8790 - val_loss: 0.3153
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 392ms/step - accuracy: 0.9142 - loss: 0.2248 - val_accuracy: 0.8775 - val_loss: 0.3334
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 395ms/step - accuracy: 0.9259 - loss: 0.1915 - val_accuracy: 0.8754 - val_loss: 0.3206
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 401ms/step - accuracy: 0.9335 - loss: 0.1762 - val_accuracy: 0.8464 - val_loss: 0.3630
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 397ms/step - accuracy: 0.9375 -

**Model Evaluation**

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 110ms/step - accuracy: 0.8718 - loss: 0.3624
Test Loss: 0.3543994128704071
Test Accuracy: 0.8741000294685364


**Building a Predictive System**

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
# example usage
new_review = "t "
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
The sentiment of the review is: negative


In [ ]:
# example usage
new_review = "As far as films go, this is likable enough. Entertaining characters, good dialogue, interesting enough story. I would have really quite liked it had I not been irritated immensely whilst watching at the utter disrespect it shows the city it is set in.<br /><br />Glasgow. In Scotland. Yet every character is English (save for Sean's girlfriend, who is Dutch). Scottish accents are heard only fleetingly in menial jobs & roles. As a Scottish woman (& as a viewer who likes her real life films to be a bit more like real life) I really don't think it would have hurt to use any one of the countless talented Scottish actors...or at least got English ones who could toss together a decent accent! The futile attempt at using the word  a few times did nothing but to further the insult."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
The sentiment of the review is: negative


In [ ]:
# example usage
new_review = "Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
The sentiment of the review is: negative
